In [1]:
import numpy as np
import pandas as pd
import networkx as nx
from pathlib import Path
from ase.io import read
from mbeml.featurization import (
    generate_ligand_racs_names,
    generate_standard_racs_names,
    sort_connecting_atoms,
    featurize_single_ligand,
)
from molSimplify.Classes.mol3D import mol3D
from molSimplify.Classes.mol2D import Mol2D
from molSimplify.Classes.ligand import ligand_breakdown, ligand_assign_consistent
from molSimplify.Informatics.lacRACAssemble import get_descriptor_vector

In [2]:
raw_data_path = Path("../../raw_data")
data_path = Path("../../data")
data_sets = {
    "training": pd.read_csv(raw_data_path / "training" / "training_data_raw.csv"),
    "validation": pd.read_csv(raw_data_path / "validation" / "validation_data_raw.csv"),
    "ligand_test": pd.read_csv(raw_data_path / "ligand_test" / "ligand_test_data_raw.csv"),
    "composition_test": pd.read_csv(raw_data_path / "composition_test" / "composition_test_data_raw.csv"),
}

In [3]:
# dictionary assigning the molecular graph determinant to a charge per connecting atom
ligand_charges = {
    "6aef01e37dbae6d1036490f90be7fdfb": -1,  # fluoride, graph_det=18.9984
    "3dc71de014d4aa968cdd9f7c1e905feb": -2,  # sulfide, graph_det=32.065
    "481b474701a5d44d8bc0a1051f702ad0": -1,  # chloride, graph_det=35.452999999
    "8c49bdf7d036bdf3178afbdb02b59e5f": -1,  # iodide, graph_det=126.90446999
    "8991adadca2e8cc23e62278399df44a1": -1,  # hydroxyl [OH-], graph_det=-243.9154775
    "09642cb93892875b5495ff15993a6289": -1,  # cyanide, graph_det=-28133.19404
    "608a852d63f3b4f49956bae07e2b3f8c": -2,  # [O-][O-], graph_det=-65270.18935
    "247db616eed8055e98564c9c3d17734e": -1,  # NC[O-], graph_det=-967339.6416
    "1e18033c0b1c215888a09256d7639180": -1,  # ox, graph_det=138760516791
    "0dfc6836e022406f1be7b5627451d590": -0.5,  # acac, graph_det=-2.822252628e+16
    "0bcd1e34289690552dd5b876ac74361e": -0.5,  # porphyrin, graph_det=2.6410120150e+51
}

In [4]:
standard_racs_names = generate_standard_racs_names()
ligand_racs_names = generate_ligand_racs_names()

def featurize_data_set(df_raw, xyzs_dir):
    df_out = df_raw.copy()
    
    for index, row in df_out.iterrows():
        # ############################################# #
        # Use molSimplify to evaluate the standard-RACs #
        # ############################################# #
        mol = mol3D()
        # Use low spin geometry for featurization
        low_spin = 2 if row["high_spin"] % 2 == 0 else 1
        mol.readfromxyz(xyzs_dir / f"{row['name']}_{low_spin}.xyz")
        _, standard_racs = get_descriptor_vector(mol)

        liglist, ligdents, ligcons = ligand_breakdown(mol)
        (
            ax_ligand_list,
            eq_ligand_list,
            _,
            _,
            ax_con_int_list,
            eq_con_int_list,
            _,
            _,
            _,
        ) = ligand_assign_consistent(mol, liglist, ligdents, ligcons, loud=False)
        # These are somewhat difficult to calculate given that there are "fractional" ligands in the ligand_list.
        # This is accounted for by the multipling the fractional ligand charge with the number of connecting atoms in the subset
        axial_charge = sum(
            [
                ligand_charges.get(Mol2D.from_mol3d(lig.mol).graph_hash(), 0) * len(con)
                for lig, con in zip(ax_ligand_list, ax_con_int_list)
            ]
        )
        equatorial_charge = sum(
            [
                ligand_charges.get(Mol2D.from_mol3d(lig.mol).graph_hash(), 0) * len(con)
                for lig, con in zip(eq_ligand_list, eq_con_int_list)
            ]
        )
        standard_racs[1] = axial_charge / len(ax_ligand_list)
        standard_racs[3] = equatorial_charge / len(eq_ligand_list)

        df_out.loc[index, standard_racs_names] = standard_racs
    

        # #################### #
        # evaluate ligand-RACs #
        # #################### #
        ase_atoms = read(xyzs_dir / f"{row['name']}_{low_spin}.xyz")
        mol2D = Mol2D.from_mol3d(mol)
        # Check that there is only one metal and that it is the first atom
        assert mol2D.find_metal() == [0]

        connecting_atoms = sort_connecting_atoms(ase_atoms, list(mol2D.neighbors(0)))

        split_graph = mol2D.copy()
        # Remove edges from center (index 0) to the connecting atoms
        split_graph.remove_edges_from([(0, c) for c in connecting_atoms])
        ligand_graphs = [split_graph.subgraph(gi) for gi in nx.connected_components(split_graph)][1:]
        
        ligand_racs = np.zeros((6, 33))
        for ci, c in enumerate(connecting_atoms):
            ligand_racs[ci, 1:] = featurize_single_ligand(split_graph, c, depth=3)

            # Kazy way of finding correct subgraph for the connecting atom c
            for lig_graph in ligand_graphs:            
                if c in lig_graph:
                    break
            else:
                raise ValueError(f'Unable to find ligand corresponding to connecting atom {c}')
            
            lig_hash = lig_graph.graph_hash()
            if lig_hash in ligand_charges:
                ligcharge = ligand_charges[lig_hash]
            else:
                ligcharge = 0.0
            ligand_racs[ci, 0] = ligcharge
            if lig_graph.graph_determinant() in ligand_charges:
                print(lig_graph.graph_determinant(), lig_graph.graph_hash())
        
        df_out.loc[index, ligand_racs_names] = ligand_racs.flatten()
    return df_out

In [5]:
for key, df_raw in data_sets.items():
    df = featurize_data_set(df_raw, raw_data_path / key / "xyzs")
    df.to_csv(data_path / f"{key}_data.csv")

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is 08/16/2024 12:39, XYZ structure generated by mol3D Class, molSimplify)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is 08/16/2024 12:39, XYZ structure generated by mol3D Class, molSimplify)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is 08/16/2024 12:39, XYZ structure generated by mol3D Class, molSimplify)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is 08/16/2024 12:39, XYZ structure generated by mol3D Class, molSimplify)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is 08/16/2024 12:39, XYZ structure generated by mol3D Class, molSimplify)

*** Open Babel Warning  in Perceive